In [1]:
# %load train.py

import tensorflow as tf
import numpy as np
import os
import cv2
import dense_net
import anchor_op
from xml_op import *
import matplotlib.pyplot as plt
%matplotlib inline

anchor_num = 3
a_num = 1
ratio_num = 3
stride=np.array([7,14,28,56])
stride_size = [32,16,8,4]
anchor_size=np.array([64,56,16,8])
ratio = np.array([[1,1],[1,2],[2,1]])
CLASS_NUM = 2
get_best_anchor_num = 300

rpn_ckpts = './checkpoint1_dir'
rpn_adam_meta = './checkpoint1_dir/MyModel'

roi_ckpts = './roi_ck1'
roi_adam_meta = './roi_ck1/MyModel'
   

def check_enviroment(path):
    if(not os.path.exists(path)):
        os.mkdir(path)
        print('create dir')
    else:
        print('already exist')
        
def show_img(img):
    plt.imshow(img)
    plt.show()

def cal_l1_loss(real_box_dx,pre_box,is_real_box,ll1,ll2,ll3):
    j1 = tf.add(real_box_dx,(-1)*pre_box)
    j2 = tf.cast(tf.less(tf.abs(j1),ll1),tf.float32)
    j3 = is_real_box*((j2)*(ll2*j1*j1)+(1.0-j2)*(tf.abs(j1)-ll3))
    j4 = tf.reduce_sum(j3)
    return j4,j3

def region_proposal(anchor_ut,sort_id,sort_score,dx_box_loc,r_box_loc,cls):
    rpn_anchor = anchor_ut.get_best_anchor(sort_id,sort_score,dx_box_loc)
    #print(rpn_anchor)
    rpn_anchor_index = anchor_ut.py_cpu_nms(rpn_anchor,get_best_anchor_num)
    dx_box_loc = rpn_anchor[rpn_anchor_index,0:4]
    #print(dx_box_loc)
    #print(dx_box_loc.shape)
    fg_bg_check,real_fg_bg_value,box_index,is_anchor = anchor_ut.collect_roi_fg_bg(dx_box_loc,r_box_loc,cls,pos_iou=0.6,neg_iou=0.1,max_anchor=64)
    #print(fg_bg_check)
    #print(real_fg_bg_value)
    #print(is_anchor)
    rpn_box = np.zeros([rpn_anchor_index.shape[0],6])
    for i in range(fg_bg_check.shape[1]):
        if(fg_bg_check[0][i]!=0):
            x1,y1,x2,y2 = dx_box_loc[i]
            rpn_box[i,:4] = x1,y1,x2,y2
            
            w = x2-x1
            h = y2-y1
            feature_num = 3+int(np.log2((np.sqrt(w*h)/224)))
            if(feature_num<0):
                feature_num = 0
            if(real_fg_bg_value[0][i][0]==1):
                rpn_box[i][4] = feature_num
                rpn_box[i][5] = 0    
            else:
                rpn_box[i][4] = feature_num
                rpn_box[i][5] = 1
            
    #print(rpn_box[:,5])
    real_fg_bg_value = np.squeeze(real_fg_bg_value)
    fg_bg_check = np.squeeze(fg_bg_check)
    #print(real_fg_bg_value.shape)
    return rpn_box,fg_bg_check,real_fg_bg_value,box_index,is_anchor

def rpn_train():
    train_data = read_xml('./outputs')
    #train_data = read_xml('./xml_test')
    '''
    img = cv2.imread('test.jpg')
    r_img = np.reshape(img,(-1,224,224,3))
    r_box = np.reshape(np.array([[78,54,148,173]],dtype=np.float),(-1,4))
    '''
    ds = dense_net.Dense_net(224,3,10,24,0.5)
    x = ds.x
    
    pre_fg,pre_fg_score = ds.all_fg,ds.all_fg_score
    pre_box_dx = ds.all_box
    
    sort_fg_index,sort_fg_score = ds.sort_fg_score()
    
    real_fg = tf.placeholder(tf.float32,[None,None,2])
    is_real_fg = tf.placeholder(tf.float32,[None,None])
    fg_num = tf.placeholder(tf.float32)
    
    real_box_dx = tf.placeholder(tf.float32,[None,None,4])
    is_real_box_dx = tf.placeholder(tf.float32,[None,None,4])
    box_num = tf.placeholder(tf.float32)
    
    fg_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
                                    labels=tf.argmax(real_fg,axis=2),logits=pre_fg)

    fg_loss = tf.reduce_sum(tf.multiply(fg_loss,is_real_fg))
    fg_loss = fg_loss/fg_num
    box_loss,j1 = cal_l1_loss(real_box_dx,pre_box_dx,is_real_box_dx,1.0,0.5,0.5)
    box_loss = (box_loss/box_num)*5
    box_loss = tf.clip_by_value(box_loss,1e-10,box_loss)
    rpn_total_loss = fg_loss+box_loss
    
    global_step = tf.get_variable('global_step', [], dtype=tf.int32,
                                  initializer=tf.constant_initializer(0), trainable=False)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.name_scope('opt'):
        #opt = tf.train.AdamOptimizer(1e-3,name='optimizer')
        #opt = tf.train.GradientDescentOptimizer(5e-3,name='optimizer')
        opt = tf.train.MomentumOptimizer(1e-4,0.9,name='optimizer')
        with tf.control_dependencies(update_ops):
            grads = opt.compute_gradients(rpn_total_loss)
            train_op = opt.apply_gradients(grads, global_step=global_step)
        #tf.summary.scalar('opti', opt)
    
    anchor_ut = anchor_op.anchor_util(stride,anchor_size,stride_size)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        ########  save
        var_list = tf.trainable_variables()
        if global_step is not None:
            var_list.append(global_step)
        g_list = tf.global_variables()
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        var_list += bn_moving_vars
        saver = tf.train.Saver(var_list=var_list,max_to_keep=5)
        ########
        
        if tf.train.latest_checkpoint(rpn_ckpts) is not None:
            saver.restore(sess, tf.train.latest_checkpoint(rpn_ckpts))
            #saver = tf.train.import_meta_graph('./checkpoint1_dir/MyModel-3250.meta')
        else:
            assert 'can not find checkpoint folder path!'
        
        
        for i in range(10000):
            print('-----')
            
            ba_x = i%86
            print(train_data[ba_x])
            img,r_box,_ = decode_xml(train_data[ba_x])
            r_img = img[np.newaxis,:,:,:]
            
            #print(r_img.shape)
            
            pf_value,pf_score,dx_box,sort_id,sort_score,td = sess.run([pre_fg,pre_fg_score,pre_box_dx,
                                                 sort_fg_index,sort_fg_score,ds.dense4],feed_dict={x:r_img})
    
            dx_box_loc = anchor_ut.get_dx_box(dx_box[0])
            
            fg_bg_check,real_fg_bg_value,box_index,is_anchor = anchor_ut.collect_fg_bg(dx_box_loc,r_box)
            
            dx_anchor_value,is_dx_box,is_box = anchor_ut.collect_anchor_dx(dx_box,box_index,real_fg_bg_value,r_box)
                          
            
            fg_ls,box_ls,fg_rank,_= sess.run([fg_loss,box_loss,sort_fg_index,train_op],feed_dict={x:r_img,
                                               real_box_dx:dx_anchor_value,is_real_box_dx:is_dx_box,box_num:is_box,
                                               real_fg:real_fg_bg_value,is_real_fg:fg_bg_check,fg_num:is_anchor
                                               })
                    
            if(i%1==0):
                for j in range(pf_score.shape[1]):
                    if(real_fg_bg_value[0][j][0]==1):
                        rank = 0
                        for k in range(fg_rank.shape[0]):
                            if(fg_rank[k]==j):
                                rank = k
                        #print('high',pf_score[0][fg_rank[0]])
                        #print('low',pf_score[0][fg_rank[-1]])
                        print('fg_score',pf_score[0][j],'rank',rank,' id',j)
                        print(fg_ls,box_ls)
                        
            if(i%50==0 and i!=0):
                saver.save(sess,rpn_adam_meta,global_step=i) 
                
def roi_train():
    train_data = read_xml('./outputs')
    '''
    img = cv2.imread('test.jpg')
    r_img = np.reshape(img,(-1,224,224,3))
    r_box = np.reshape(np.array([[78,54,148,173],[28,12,64,99]],dtype=np.float),(-1,4))
    '''
    ds = dense_net.Dense_net(224,3,10,24,0.5)
    x = ds.x
    
    pre_fg,pre_fg_score = ds.all_fg,ds.all_fg_score
    pre_box_dx = ds.all_box
    
    sort_fg_index,sort_fg_score = ds.sort_fg_score()
       
    roi_cls_score,roi_cls_score_p,roi_box_dx = ds.cls_score,ds.cls_score_p,ds.final_box
    #print(roi_cls_score)
    roi_feature_map = ds.roi_feature_map
    roi_cls = tf.placeholder(tf.float32,[None,CLASS_NUM])
    roi_is_fg = tf.placeholder(tf.float32,[None])
    roi_fg_num = tf.placeholder(tf.float32)
    '''
    rpn_boxes = tf.placeholder(tf.float32,[None,4])
    rpn_num = tf.placeholder(tf.int32)
    roi_feature_2 = ds.roi_pooling_2(rpn_boxes,rpn_num)
    print(roi_feature_2)
    '''
    real_box_dx = tf.placeholder(tf.float32,[None,None,4])
    is_real_box_dx = tf.placeholder(tf.float32,[None,None,4])
    box_num = tf.placeholder(tf.float32)
    
    box_loss,j1 = cal_l1_loss(real_box_dx,ds.final_box,is_real_box_dx,1.0,0.5,0.5)
    box_loss = (box_loss/box_num)
    box_loss = tf.clip_by_value(box_loss,1e-10,box_loss)*5


    cls_loss =  tf.nn.sparse_softmax_cross_entropy_with_logits(
                                    labels=tf.argmax(roi_cls,axis=1),logits=roi_cls_score)
    cls_loss = tf.reduce_sum(tf.multiply(cls_loss,roi_is_fg))
    cls_loss = cls_loss/roi_fg_num
    
    roi_total_loss = cls_loss+box_loss
    
    global_step = tf.get_variable('global_step', [], dtype=tf.int32,
                                  initializer=tf.constant_initializer(0), trainable=False)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    with tf.name_scope('roi_opt'):
        #opt = tf.train.AdamOptimizer(1e-3,name='optimizer')
        #opt = tf.train.GradientDescentOptimizer(5e-3,name='optimizer')
        opt = tf.train.MomentumOptimizer(1e-4,0.9,name='optimizer')
        with tf.control_dependencies(update_ops):
            grads = opt.compute_gradients(roi_total_loss)
            train_op = opt.apply_gradients(grads, global_step=global_step)
        #tf.summary.scalar('opti', opt)
    
    anchor_ut = anchor_op.anchor_util(stride,anchor_size,stride_size)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        ########  saves
        var_list = tf.trainable_variables()
        if global_step is not None:
            var_list.append(global_step)
        g_list = tf.global_variables()
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        var_list += bn_moving_vars
        saver = tf.train.Saver(var_list=var_list,max_to_keep=5)
        ########
        
        if tf.train.latest_checkpoint(rpn_ckpts) is not None:
            #saver.restore(sess, tf.train.latest_checkpoint(rpn_ckpts))
            saver.restore(sess, tf.train.latest_checkpoint(roi_ckpts))
        else:
            assert 'can not find checkpoint folder path!'
        
        
        for i in range(1000):
            print('-----')
            ba_x = i%len(train_data)
            #print(train_data[ba_x])
            img,r_box,cls = decode_xml(train_data[ba_x])
            r_img = img[np.newaxis,:,:,:]
            
            pf_value,pf_score,dx_box,sort_id,sort_score,g_step = sess.run([pre_fg,pre_fg_score,pre_box_dx,
                                                 sort_fg_index,sort_fg_score,global_step],feed_dict={x:r_img})
    
            dx_box_loc = anchor_ut.get_dx_box(dx_box[0])
            #print(dx_box_loc)
            if(i<=300):
                rpn_box,roi_fg_bg_check,roi_real_fg_bg_value,roi_box_index,roi_is_anchor = region_proposal(anchor_ut,sort_id,sort_score,dx_box_loc,r_box,cls)
                #print(roi_real_fg_bg_value)
                roi_feature,cls = ds.roi_pooling(rpn_box)
                r_feature = sess.run(roi_feature,feed_dict={x:r_img})

                rcs,rcsp,rdx = sess.run([roi_cls_score,roi_cls_score_p,roi_box_dx],feed_dict={roi_feature_map:r_feature})
                #print(rdx[0],rpn_box[0],roi_box_dx)

                dx_anchor_value,is_dx_box,is_box = anchor_ut.collect_anchor_dx(rpn_box[np.newaxis,:,:4],
                                                                        roi_box_index,roi_real_fg_bg_value[np.newaxis,:,:],r_box,rpn_box[:,:4])
                print(is_box)
            else:
                ####
                rpn_box = anchor_ut.region_proposal_2(sort_id,sort_score,dx_box_loc)
                roi_feature = ds.roi_pooling_1(rpn_box)
                r_feature = sess.run(roi_feature,feed_dict={x:r_img})
                rcs,rcsp,rdx = sess.run([roi_cls_score,roi_cls_score_p,roi_box_dx],feed_dict={roi_feature_map:r_feature})
                #print(rdx[0],rpn_box[0])
                dx_box_loc2 = anchor_ut.get_dx_box(rdx,rpn_box[:,:4])
                roi_fg_bg_check,roi_real_fg_bg_value,roi_box_index,roi_is_anchor = anchor_ut.collect_roi_fg_bg(dx_box_loc2,r_box,cls,pos_iou=0.6,neg_iou=0.1,max_anchor=128)
                dx_anchor_value,is_dx_box,is_box = anchor_ut.collect_anchor_dx(dx_box_loc2,
                                                                        roi_box_index,roi_real_fg_bg_value,r_box,rpn_box[:,:4])
                roi_real_fg_bg_value = np.reshape(roi_real_fg_bg_value,(-1,CLASS_NUM))
                roi_fg_bg_check = roi_fg_bg_check[0]
                print(is_box)
                ####
            
            '''
            #print(rpn_box)
            #r_feature = sess.run(roi_feature,feed_dict={rpn_boxes:rpn_box[np.newaxis,:,:],rpn_num:rpn_box.shape[0]})
            r = sess.run(roi_feature_2,feed_dict={rpn_boxes:rpn_box[:,:4],rpn_num:rpn_box.shape[0],x:r_img})
            print(r.shape,r[0][0][0][0:10])
            #print(dsf)
            '''
            for j in range(roi_real_fg_bg_value.shape[0]):
                if(roi_real_fg_bg_value[j][0]==1):
                    print('is fg',rcsp[j])
                '''     
                    im = img.copy()
                    x1,y1,x2,y2 = rpn_box[j,:4]

                        cv2.rectangle(im,(int(x1),int(y1)),(int(x2),int(y2)), (0, 255, 0), 2)
                        show_img(im)
                else:
                    print(rcsp[j])
                    cv2.rectangle(im,(int(x1),int(y1)),(int(x2),int(y2)), (0, 255, 0), 2)
                    show_img(im)
                '''   
            
            #print(roi_is_anchor)
            #print(roi_real_fg_bg_value.shape,roi_fg_bg_check.shape,roi_is_anchor)
            #print(is_dx_box.shape,dx_anchor_value.shape,is_dx_box.shape,is_box)
            cls_ls,box_ls,_ = sess.run([cls_loss,box_loss,train_op],
                                        feed_dict={roi_cls:roi_real_fg_bg_value,roi_is_fg:roi_fg_bg_check,
                                           roi_fg_num:roi_is_anchor,roi_feature_map:r_feature,x:r_img,
                                           real_box_dx:dx_anchor_value,is_real_box_dx:is_dx_box,box_num:is_box})
            
            print('step:',i,cls_ls,box_ls)
                        
            if(i%10==0 and i!=0):
                saver.save(sess,roi_adam_meta,global_step=g_step) 


if(__name__=='__main__'):
    #rpn_train()
    roi_train()

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
Tensor("dense_net/resnext/max_pooling2d/MaxPool:0", shape=(?, 56, 56, 64), dtype=float32)
Tensor("dense_net/resnext/Relu_17:0", shape=(?, 56, 56, 64), dtype=float32)
Tensor("dense_net/resnext/Relu_34:0", shape=(?, 28, 28, 128), dtype=float32)
Tensor("dense_net/resnext/Relu_51:0", shape=(?, 14, 14, 256), dtype=float32)
Tensor("dense_net/resnext/Relu_68:0", shape=(?, 7, 7, 512), dtype=float32)
(<tf.Tensor 'concat:0' shape=(?, 12495, 2) dtype=float32>, <tf.Tensor 'concat_1:0' shape=(?, 12495, 2) dtype=float32>, <tf.Tensor 'concat_2:0' shape=(?, 12495, 4) dtype=float32>)
Tensor("flatten/Reshape:0", shape=(?, 12544), dtype=float32)
(?, 12495, 2)
INFO:tensorflow:Restoring parameters from ./roi_ck1/MyModel-1632
-----
0.875
0.9119914916245679
0.6061120543293718
0.9168671478214381
0.9423076923076923
0.9222133119486768
0.9482525060959089
(7, 57)
7
('is fg', array([0.9979804 , 0.00201963], dtype=float32))
('is fg', array([0.

0.9352750809061489
(12, 52)
12
('is fg', array([0.9672192, 0.0327808], dtype=float32))
('is fg', array([0.96136713, 0.0386328 ], dtype=float32))
('is fg', array([0.9007411 , 0.09925894], dtype=float32))
('is fg', array([0.9532497 , 0.04675034], dtype=float32))
('is fg', array([0.9400763 , 0.05992374], dtype=float32))
('is fg', array([0.92882603, 0.07117397], dtype=float32))
('is fg', array([0.9672192, 0.0327808], dtype=float32))
('is fg', array([0.944903, 0.055097], dtype=float32))
('is fg', array([0.96049577, 0.03950425], dtype=float32))
('is fg', array([0.90831274, 0.09168722], dtype=float32))
('is fg', array([0.94745046, 0.05254949], dtype=float32))
('is fg', array([0.9272108, 0.0727892], dtype=float32))
('step:', 9, 0.53664005, 0.0053292005)
-----
0.9118122977346278
0.8828828828828829
0.9301698571043407
0.9004237288135594
0.6018518518518519
0.9470588235294117
0.9470588235294117
0.8535564853556485
0.8771186440677966
0.9349593495934959
0.9433962264150944
0.9047365620010644
0.91743119

(12, 52)
12
('is fg', array([0.92097837, 0.07902165], dtype=float32))
('is fg', array([0.76760525, 0.23239474], dtype=float32))
('is fg', array([0.9197558 , 0.08024421], dtype=float32))
('is fg', array([0.921392  , 0.07860799], dtype=float32))
('is fg', array([0.8955743 , 0.10442572], dtype=float32))
('is fg', array([0.89734   , 0.10265996], dtype=float32))
('is fg', array([0.7951204 , 0.20487955], dtype=float32))
('is fg', array([0.91927755, 0.08072248], dtype=float32))
('is fg', array([0.92081684, 0.07918321], dtype=float32))
('is fg', array([0.9289152 , 0.07108472], dtype=float32))
('is fg', array([0.90155417, 0.09844583], dtype=float32))
('is fg', array([0.8903333 , 0.10966668], dtype=float32))
('step:', 20, 0.49324185, 0.0058708703)
-----
0.8543101074084274
0.8910582908885116
0.9146757679180887
0.8664804469273742
0.8625034237195289
0.8667767685108726
0.8561111111111112
0.8209774848984075
(8, 56)
8
('is fg', array([0.8910722 , 0.10892778], dtype=float32))
('is fg', array([0.9018789

(14, 50)
14
('is fg', array([0.8438775, 0.1561225], dtype=float32))
('is fg', array([0.9189402 , 0.08105983], dtype=float32))
('is fg', array([0.9243296 , 0.07567043], dtype=float32))
('is fg', array([0.8438775, 0.1561225], dtype=float32))
('is fg', array([0.910327  , 0.08967294], dtype=float32))
('is fg', array([0.92126775, 0.07873228], dtype=float32))
('is fg', array([0.8551844 , 0.14481567], dtype=float32))
('is fg', array([0.90731853, 0.09268146], dtype=float32))
('is fg', array([0.9073976 , 0.09260233], dtype=float32))
('is fg', array([0.9239346 , 0.07606542], dtype=float32))
('is fg', array([0.87086964, 0.12913041], dtype=float32))
('is fg', array([0.65700585, 0.34299415], dtype=float32))
('is fg', array([0.89626026, 0.10373972], dtype=float32))
('is fg', array([0.6104275 , 0.38957256], dtype=float32))
('step:', 32, 0.60288554, 0.04472805)
-----
0.9348837209302325
0.8776667566837699
0.9389013452914798
0.912106135986733
0.9577464788732394
0.9667582417582418
0.6146572104018913
0.95

...

9
('is fg', array([0.9144656 , 0.08553442], dtype=float32))
('is fg', array([0.8825114 , 0.11748857], dtype=float32))
('is fg', array([0.9232237 , 0.07677627], dtype=float32))
('is fg', array([0.8945111 , 0.10548894], dtype=float32))
('is fg', array([0.955081  , 0.04491903], dtype=float32))
('is fg', array([0.5355577 , 0.46444228], dtype=float32))
('is fg', array([0.84966135, 0.15033868], dtype=float32))
('is fg', array([0.95435417, 0.04564586], dtype=float32))
('is fg', array([0.84966135, 0.15033868], dtype=float32))
('step:', 73, 0.53646016, 0.022647213)
-----
0.9630642954856361
0.9335717935615738
0.9442573256988885
0.9628996409642674
0.6008081371046399
0.6171808661194452
(6, 58)
6
('is fg', array([0.945443  , 0.05455706], dtype=float32))
('is fg', array([0.8530763 , 0.14692375], dtype=float32))
('is fg', array([0.8416985 , 0.15830149], dtype=float32))
('is fg', array([0.94944364, 0.05055632], dtype=float32))
('is fg', array([0.88004065, 0.11995938], dtype=float32))
('is fg', array([

8
('is fg', array([0.96771693, 0.03228306], dtype=float32))
('is fg', array([0.97741836, 0.02258165], dtype=float32))
('is fg', array([0.9848739 , 0.01512614], dtype=float32))
('is fg', array([0.95650935, 0.04349064], dtype=float32))
('is fg', array([0.9710824 , 0.02891755], dtype=float32))
('is fg', array([0.98061395, 0.01938606], dtype=float32))
('is fg', array([0.9851354 , 0.01486466], dtype=float32))
('is fg', array([0.9572211 , 0.04277891], dtype=float32))
('step:', 85, 0.31157997, 0.0066032745)
-----
0.875
0.9119914916245679
0.6061120543293718
0.9168671478214381
0.9423076923076923
0.9222133119486768
0.9482525060959089
(7, 57)
7
('is fg', array([0.99175113, 0.00824886], dtype=float32))
('is fg', array([0.9785584 , 0.02144159], dtype=float32))
('is fg', array([0.98489594, 0.01510402], dtype=float32))
('is fg', array([0.98544276, 0.01455723], dtype=float32))
('is fg', array([0.99310315, 0.00689685], dtype=float32))
('is fg', array([0.9857885 , 0.01421149], dtype=float32))
('is fg', 

(13, 51)
13
('is fg', array([0.96233135, 0.03766859], dtype=float32))
('is fg', array([0.97672063, 0.02327934], dtype=float32))
('is fg', array([0.95902455, 0.04097549], dtype=float32))
('is fg', array([0.9844797 , 0.01552027], dtype=float32))
('is fg', array([0.96978164, 0.03021839], dtype=float32))
('is fg', array([0.956824  , 0.04317599], dtype=float32))
('is fg', array([0.97672063, 0.02327934], dtype=float32))
('is fg', array([0.96258855, 0.03741149], dtype=float32))
('is fg', array([0.95685697, 0.04314301], dtype=float32))
('is fg', array([0.9820057 , 0.01799433], dtype=float32))
('is fg', array([0.9515403 , 0.04845966], dtype=float32))
('is fg', array([0.9631308 , 0.03686919], dtype=float32))
('is fg', array([0.6791376, 0.3208624], dtype=float32))
('step:', 96, 0.3810717, 0.025843356)
-----
0.9530090684253916
0.9345794392523364
0.9615384615384616
0.9345794392523364
0.9857142857142858
0.9857142857142858
0.9523809523809523
0.9523809523809523
(8, 56)
8
('is fg', array([0.9918942 , 0

8
('is fg', array([0.8522614 , 0.14773859], dtype=float32))
('is fg', array([0.8773233 , 0.12267662], dtype=float32))
('is fg', array([0.76046294, 0.23953708], dtype=float32))
('is fg', array([0.93859404, 0.06140598], dtype=float32))
('is fg', array([0.9495669 , 0.05043305], dtype=float32))
('is fg', array([0.7943371 , 0.20566286], dtype=float32))
('is fg', array([0.801993, 0.198007], dtype=float32))
('is fg', array([0.7283146 , 0.27168536], dtype=float32))
('step:', 107, 0.64554167, 0.008398035)
-----
0.966657326982348
0.9394860458690246
0.9530090684253916
0.9345794392523364
0.9475896860986547
0.9607843137254902
0.9345794392523364
0.9128065395095368
(8, 56)
8
('is fg', array([0.98978066, 0.01021929], dtype=float32))
('is fg', array([0.9780245 , 0.02197556], dtype=float32))
('is fg', array([0.9780377 , 0.02196228], dtype=float32))
('is fg', array([0.9763282, 0.0236718], dtype=float32))
('is fg', array([0.98139143, 0.01860862], dtype=float32))
('is fg', array([0.9907391, 0.0092609], dty

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
tf.gather?

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
